# Cleaning WMDA Data

In [119]:

import pandas as pd
import numpy as np

#%matplotlib notebook to make interactive within same window as JN
#%matplotlib
#%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D

#SciKit Learn
from sklearn import svm
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

from sklearn import preprocessing
from sklearn.decomposition import PCA

#Plotly
import plotly.express as px
import plotly.graph_objects as go

import seaborn as sns
import scipy as sci
import statsmodels as stats



In [120]:
# Get .csv Data and change missing values label to NaN (was "..")
WorldData = pd.read_csv("WorldMarketData.csv", na_values = '..')

In [121]:
# Turn data into pandas DataFrame
df_original = pd.DataFrame(WorldData)

In [122]:
df_original.head(5)

,Country Name,Country Code,Series Name,Series Code,1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,United States,USA,Agricultural raw materials exports (% of merch...,TX.VAL.AGRI.ZS.UN,3.49,3.86,3.07,2.67,2.35,2.08,...,2.31,2.63,2.80,2.45,2.42,2.30,2.27,2.30,2.31,2.26
1,United States,USA,Agricultural raw materials imports (% of merch...,TM.VAL.AGRI.ZS.UN,2.04,2.11,2.04,1.91,1.73,1.66,...,0.85,0.96,0.98,0.88,0.91,0.93,0.91,0.95,0.91,0.88
2,United States,USA,Average time to clear exports through customs ...,IC.CUS.DURS.EX,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,United States,USA,Bribery incidence (% of firms experiencing at ...,IC.FRM.BRIB.ZS,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,United States,USA,Changes in inventories (current US$),NE.GDI.STKB.CD,"63,785,000,000.00","31,174,000,000.00","30,799,000,000.00","70,927,000,000.00","63,719,000,000.00","60,816,000,000.00",...,"-150,772,000,000.00","53,917,000,000.00","46,262,000,000.00","71,212,000,000.00","104,531,000,000.00","84,490,000,000.00","128,724,000,000.00","28,940,000,000.00","25,484,000,000.00",nan


In [123]:
# Display only two decimal places for Pandas by changing the display options
pd.options.display.float_format = '{:,.2f}'.format

In [124]:
# Change Column Labels accordignly
df_original.columns = ['country', 'country_code', 'series', 'series_code', '1994', '1995', '1996', '1997', '1998', 
             '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', 
             '2012', '2013', '2014', '2015', '2016', '2017', '2018']

In [125]:
# List the original Columns
df_original.columns[:]

Index(['country', 'country_code', 'series', 'series_code', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018'],
      dtype='object')

In [126]:
# Get the overhead info on the data
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3805 entries, 0 to 3804
Data columns (total 29 columns):
country         3802 non-null object
country_code    3800 non-null object
series          3800 non-null object
series_code     3800 non-null object
1994            1654 non-null float64
1995            1733 non-null float64
1996            1824 non-null float64
1997            1827 non-null float64
1998            1841 non-null float64
1999            1867 non-null float64
2000            1898 non-null float64
2001            1926 non-null float64
2002            1945 non-null float64
2003            2061 non-null float64
2004            2112 non-null float64
2005            2263 non-null float64
2006            2371 non-null float64
2007            2803 non-null float64
2008            2365 non-null float64
2009            2366 non-null float64
2010            2834 non-null float64
2011            2393 non-null float64
2012            2819 non-null float64
2013            2428 no

In [127]:
# Drop useless columns and change the dataframe name to "df"
df = df_original.drop(columns=['country_code', 'series_code'])
df.head()

,country,series,1994,1995,1996,1997,1998,1999,2000,2001,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,United States,Agricultural raw materials exports (% of merch...,3.49,3.86,3.07,2.67,2.35,2.08,2.27,2.31,...,2.31,2.63,2.80,2.45,2.42,2.30,2.27,2.30,2.31,2.26
1,United States,Agricultural raw materials imports (% of merch...,2.04,2.11,2.04,1.91,1.73,1.66,1.38,1.35,...,0.85,0.96,0.98,0.88,0.91,0.93,0.91,0.95,0.91,0.88
2,United States,Average time to clear exports through customs ...,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,United States,Bribery incidence (% of firms experiencing at ...,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,United States,Changes in inventories (current US$),"63,785,000,000.00","31,174,000,000.00","30,799,000,000.00","70,927,000,000.00","63,719,000,000.00","60,816,000,000.00","54,544,000,000.00","-38,284,000,000.00",...,"-150,772,000,000.00","53,917,000,000.00","46,262,000,000.00","71,212,000,000.00","104,531,000,000.00","84,490,000,000.00","128,724,000,000.00","28,940,000,000.00","25,484,000,000.00",nan


In [128]:
df.tail()

,country,series,1994,1995,1996,1997,1998,1999,2000,2001,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
3800,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3801,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3802,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3803,Data from database: World Development Indicators,NaN,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3804,Last Updated: 07/10/2019,NaN,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [129]:
#Get rid of the bottom rows that have no value/meaning
df = df.drop(df.index[3800:])
df.tail(5)

,country,series,1994,1995,1996,1997,1998,1999,2000,2001,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
3795,High income,"Time to export, border compliance (hours)",nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,28.05,26.81,26.54,26.42,26.05
3796,High income,"Time to import, border compliance (hours)",nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,30.05,27.81,27.78,27.40,27.19
3797,High income,"Time to export, documentary compliance (hours)",nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,15.93,15.10,14.86,13.45,13.08
3798,High income,"Time to import, documentary compliance (hours)",nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,21.67,20.86,19.24,18.67,17.71
3799,High income,Trade (% of GDP),41.45,43.77,44.32,46.35,46.64,46.96,51.53,50.30,...,53.12,58.39,62.45,62.90,62.83,62.92,61.38,59.66,61.56,nan


# Tidying Dataframe

### Make the dataframe more interpretable for modeling

Many models and algorithms use a very particular structure. Often it is best to organize data in a way that agrees with this. Below are two sources I found that describe some of the usefulness behind this, for further reading.

- [Tidy Data Paper](http://vita.had.co.nz/papers/tidy-data.pdf)
- [Tidy Data Logic](https://www.jeannicholashould.com/tidy-data-in-python.html)

>Essentially, a data layout that may not be as intuitive for us visually, may be far simpler and more efficient from a computational standpoint.

In [130]:
df.head()

,country,series,1994,1995,1996,1997,1998,1999,2000,2001,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,United States,Agricultural raw materials exports (% of merch...,3.49,3.86,3.07,2.67,2.35,2.08,2.27,2.31,...,2.31,2.63,2.80,2.45,2.42,2.30,2.27,2.30,2.31,2.26
1,United States,Agricultural raw materials imports (% of merch...,2.04,2.11,2.04,1.91,1.73,1.66,1.38,1.35,...,0.85,0.96,0.98,0.88,0.91,0.93,0.91,0.95,0.91,0.88
2,United States,Average time to clear exports through customs ...,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,United States,Bribery incidence (% of firms experiencing at ...,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,United States,Changes in inventories (current US$),"63,785,000,000.00","31,174,000,000.00","30,799,000,000.00","70,927,000,000.00","63,719,000,000.00","60,816,000,000.00","54,544,000,000.00","-38,284,000,000.00",...,"-150,772,000,000.00","53,917,000,000.00","46,262,000,000.00","71,212,000,000.00","104,531,000,000.00","84,490,000,000.00","128,724,000,000.00","28,940,000,000.00","25,484,000,000.00",nan


In [131]:
#Use .melt to Tidy the dataframe
df1 = pd.melt(df,
                       ["country", 'series'],
                       var_name="year",
                       value_name="value")
#df1 = df1.sort_values(by=["year"], ascending = True)
df1.head(10)

,country,series,year,value
0,United States,Agricultural raw materials exports (% of merch...,1994,3.49
1,United States,Agricultural raw materials imports (% of merch...,1994,2.04
2,United States,Average time to clear exports through customs ...,1994,nan
3,United States,Bribery incidence (% of firms experiencing at ...,1994,nan
4,United States,Changes in inventories (current US$),1994,"63,785,000,000.00"
5,United States,Commercial service exports (current US$),1994,"183,327,000,000.00"
6,United States,Commercial service imports (current US$),1994,"120,768,000,000.00"
7,United States,"Computer, communications and other services (%...",1994,36.14
8,United States,"Computer, communications and other services (%...",1994,24.38
9,United States,Cost of business start-up procedures (% of GNI...,1994,nan


### Testing Tidy Data

In [132]:
# Brief example of how it can make computation faster and more effective
dfg = df1[df1['series'] == 'GDP (current US$)']
dfg

,country,series,year,value
21,United States,GDP (current US$),1994,"7,287,236,000,000.00"
97,United Kingdom,GDP (current US$),1994,"1,140,489,745,944.29"
173,China,GDP (current US$),1994,"564,324,670,005.92"
249,Russian Federation,GDP (current US$),1994,"395,077,301,248.46"
325,Argentina,GDP (current US$),1994,"257,440,000,000.00"
401,Australia,GDP (current US$),1994,"322,208,924,247.66"
477,Bahrain,GDP (current US$),1994,"5,567,553,457.45"
553,Belgium,GDP (current US$),1994,"246,194,938,750.90"
629,Brazil,GDP (current US$),1994,"558,111,997,497.26"
705,Canada,GDP (current US$),1994,"578,139,279,437.61"


In [133]:
# Use pivot to replicate the previous format
dfg.pivot(index = 'country', columns = 'year', values = 'value')

year,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
country,,,,,,,,,,,,,,,,,,,,,
Argentina,"257,440,000,000.00","258,031,750,000.00","272,149,750,000.00","292,859,000,000.00","298,948,250,000.00","283,523,000,000.00","284,203,750,000.00","268,696,750,000.00","97,724,004,251.86","127,586,973,492.18",...,"332,976,484,577.62","423,627,422,092.49","530,163,281,574.66","545,982,375,701.13","552,025,140,252.25","526,319,673,731.64","594,749,285,413.21","557,531,376,217.97","642,695,864,756.35","518,475,134,084.36"
Australia,"322,208,924,247.66","367,216,364,716.36","400,302,731,411.23","434,568,007,512.91","398,899,138,574.24","388,608,221,581.65","415,222,633,925.77","378,376,086,723.19","394,648,911,678.53","466,488,060,570.76",...,"927,805,183,330.88","1,146,138,465,603.81","1,396,649,906,339.35","1,546,151,783,872.96","1,576,184,467,015.49","1,467,483,705,131.74","1,351,520,093,185.79","1,210,028,388,411.70","1,330,803,227,996.08","1,432,195,178,668.32"
Bahrain,"5,567,553,457.45","5,849,467,819.15","6,101,861,436.17","6,349,202,393.62","6,183,776,595.74","6,621,010,372.34","9,062,906,914.89","8,976,207,712.77","9,632,155,053.19","11,074,822,074.47",...,"22,938,218,085.11","25,713,271,276.60","28,776,595,744.68","30,749,308,510.64","32,539,547,872.34","33,387,712,765.96","31,125,851,063.83","32,250,132,978.72","35,432,686,170.21","37,746,196,808.51"
Belgium,"246,194,938,750.90","289,567,323,481.12","281,358,175,895.77","254,813,599,458.73","260,601,911,535.90","260,202,429,149.80","237,904,919,845.22","237,841,968,680.09","258,860,436,664.78","319,002,821,670.43",...,"484,552,792,442.34","483,548,031,197.11","527,008,453,886.93","497,884,216,568.87","520,925,468,952.94","530,808,391,525.11","455,940,347,170.37","469,677,213,719.19","494,901,708,704.27","531,766,935,070.73"
Brazil,"558,111,997,497.26","769,305,386,182.85","850,426,432,991.74","883,199,625,324.68","863,723,411,632.92","599,388,579,985.67","655,420,645,476.91","559,372,276,081.97","507,962,487,700.02","558,319,920,831.98",...,"1,667,019,783,585.08","2,208,871,646,202.82","2,616,201,578,192.25","2,465,188,674,415.03","2,472,806,456,101.29","2,455,994,050,148.75","1,802,214,373,741.32","1,796,275,437,087.99","2,053,594,973,992.61","1,868,626,087,908.48"
Canada,"578,139,279,437.61","604,031,623,433.40","628,546,387,972.13","652,823,920,265.78","631,813,279,406.81","676,084,000,807.70","742,295,468,318.63","736,379,777,892.56","757,950,678,646.53","892,382,413,817.71",...,"1,371,153,004,986.44","1,613,542,812,589.39","1,789,140,665,202.84","1,823,966,686,523.63","1,842,018,420,853.10","1,801,480,123,070.82","1,552,899,925,233.71","1,526,705,529,135.32","1,646,867,220,617.47","1,709,327,324,240.04"
Chile,"57,008,425,295.83","73,447,063,319.30","78,039,572,221.60","84,952,360,922.47","81,577,430,181.41","75,173,794,497.03","77,860,932,151.85","70,979,923,960.37","69,736,811,435.10","75,643,459,839.60",...,"172,389,498,444.62","218,537,551,220.07","252,251,992,029.44","267,122,320,056.70","278,384,332,694.36","260,541,637,327.86","243,919,079,437.22","250,339,876,460.23","277,746,457,909.87","298,231,139,609.45"
China,"564,324,670,005.92","734,547,898,220.51","863,746,717,503.79","961,603,952,951.82","1,029,043,097,554.08","1,093,997,267,271.06","1,211,346,869,605.24","1,339,395,718,865.30","1,470,550,015,081.55","1,660,287,965,662.68",...,"5,101,702,432,883.45","6,087,164,527,421.24","7,551,500,425,597.77","8,532,230,724,141.76","9,570,405,758,739.79","10,438,529,153,237.60","11,015,542,352,468.90","11,137,945,669,350.60","12,143,491,448,186.10","13,608,151,864,637.90"
Colombia,"81,703,500,846.04","92,507,279,383.04","97,160,109,277.81","106,659,508,271.26","98,443,739,941.17","86,186,158,684.77","99,886,577,330.73","98,211,749,595.54","97,963,003,804.79","94,641,378,693.22",...,"232,494,134,119.68","286,103,648,654.54","334,454,039,093.59","370,574,421,703.06","381,866,608,838.80","381,112,119,657.45","293,481,748,

## Back to the Primary Tidy Data

In [134]:
# Drop the following "countries" because they wildly outscale the others (since they are aggregates)
df1 = df1.set_index('country').drop(index = ['World', 'Low income', 'Middle income', 'High income']).reset_index()
df1.head()

,country,series,year,value
0,United States,Agricultural raw materials exports (% of merch...,1994,3.49
1,United States,Agricultural raw materials imports (% of merch...,1994,2.04
2,United States,Average time to clear exports through customs ...,1994,nan
3,United States,Bribery incidence (% of firms experiencing at ...,1994,nan
4,United States,Changes in inventories (current US$),1994,"63,785,000,000.00"


# Put it all together for a *one-cell-get_data_and_clean-solution*

In [135]:
import pandas as pd
import numpy as np

WorldData = pd.read_csv("WorldMarketData.csv", na_values = '..')
df_original = pd.DataFrame(WorldData)
pd.options.display.float_format = '{:,.2f}'.format
df_original.columns = ['country', 'country_code', 'series', 'series_code', '1994', '1995', '1996', '1997', '1998', 
             '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', 
             '2012', '2013', '2014', '2015', '2016', '2017', '2018']

df = df_original.drop(columns=['country_code', 'series_code'])
df = df.drop(df.index[3800:])
df1 = pd.melt(df,
                       ["country", 'series'],
                       var_name="year",
                       value_name="value")
#df1 = df1.sort_values(by=["year"], ascending = True)
df1 = df1.set_index('country').drop(index = ['World', 'Low income', 'Middle income', 'High income'])
df1 = df1.reset_index()
df1.head()

,country,series,year,value
0,United States,Agricultural raw materials exports (% of merch...,1994,3.49
1,United States,Agricultural raw materials imports (% of merch...,1994,2.04
2,United States,Average time to clear exports through customs ...,1994,nan
3,United States,Bribery incidence (% of firms experiencing at ...,1994,nan
4,United States,Changes in inventories (current US$),1994,"63,785,000,000.00"


## We now have a single cell that organizes, cleans, and restructures the WMDA data